10.2 Vector 类第一版：与Vector2d 类兼容
在这里先故意让他们不兼容

In [ ]:
from array import array
import math
import reprlib

class Vector:
    typecode = 'd'

    def __init__(self, components) -> None:
        # self._components是受保护的实例属性，把 Vector 分量保存在一个数组中
        self._components = array(self.typecode, components)
    
    def __iter__(self): # 将实例变成可迭代的对象
        return iter(self._components)
    
    def __repr__(self) -> str:
        components = reprlib.repr(self._components) # reprlib.repr() 可以让输出太多的时候使用 ... 省略一部分
        components = components[components.find('['):-1] # 先去掉前面的 array('d' 和 后面的 )
        return 'Vector{[]}'.format(components)

    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):  # 直接计算平方和
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod    # 直接传入 memoryview 给构造方法，而不是拆包
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

# 10.4  Vector 类第 2 版 ：可切片的序列
支持序列属性非常简单。只需要实现 \_\_len__ 和 \_\_getitem__ 方法

In [ ]:
class Vector:
    # 省略
    # ...

    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        return self._components[index]

In [8]:
# 完整代码
from array import array
import math
import reprlib

class Vector:
    typecode = 'd'

    def __init__(self, components) -> None:
        # self._components是受保护的实例属性，把 Vector 分量保存在一个数组中
        self._components = array(self.typecode, components)
    
    def __iter__(self): # 将实例变成可迭代的对象
        return iter(self._components)
    
    def __repr__(self) -> str:
        components = reprlib.repr(self._components) # reprlib.repr() 可以让输出太多的时候使用 ... 省略一部分
        components = components[components.find('['):-1] # 先去掉前面的 array('d' 和 后面的 )
        return 'Vector{[]}'.format(components)

    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):  # 直接计算平方和
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod    # 直接传入 memoryview 给构造方法，而不是拆包
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        return self._components[index]

In [9]:
# 添加这两个方法后，就能执行下面的操作了
v1 = Vector([3, 4, 5])
len(v1)

3

In [10]:
v1[0], v1[-1]

(3.0, 5.0)

In [12]:
v7 = Vector(range(7))
v7[1:4]

array('d', [1.0, 2.0, 3.0])

可以看到，现在连切片都支持了，不过尚不完美，如果切片返回的也是 Vector 实例就更好了。为了把 Vector 实例的切片也变成 Vector 实例，我们不能简单地委托给数组切片，我们要分析传给 __getitem__ 方法的参数，做适当的处理

## 10.4.1 切片原理 

In [14]:
# 了解 __getitem__ 和切片的行为
class MySeq:
    def __getitem__(self, index):
        return index

s = MySeq()

In [19]:
print(s[1]) # 单个索引
print(s[1:4])   # 1:4 表示法变成了 slice(1, 4, None)
print(s[1:4:2])   # slice(1, 4, 2) 表示从 1 开始，到 4 结束，步幅为 2
print(s[1:4:2, 9]) # 如果 [] 中有逗号，那么__getitem__收到的是元组
print(s[1:4:2, 7:9]) # 元组可以有多个切片对象

1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


In [20]:
# 查看 slice 类的属性
slice

slice

In [21]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

通过审查 slice，发现它有 start、stop 和 step 属性，以及 indices 方法  
indices方法有很大的作用

In [22]:
help(slice.indices)

Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



S.indices(len) -> (start, stop, stride)  
给定长度为 len 的序列，计算 S 表示的扩展切片的起始，结尾以及步幅，他会“整顿”元组以让输出的参数为正

In [23]:
slice(None, 10, 2).indices(5)

(0, 5, 2)

In [24]:
slice(-3, None, None).indices(5)

(2, 5, 1)

Vector 类中无需使用 slice.indices() 方法，因为受到切片参数时，我们会委托 _components 数组处理。但如果你没有底层序列类型作为依靠，那么使用这个方法能节省大量时间。

# 10.4.2 能处理切片的 __getitem__ 方法

In [ ]:
import numbers
class Vector:
    # 省略
    # ...

    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):     # 如果参数是 slice 对象
            return cls(self._components[index]) # 返回新的类
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be initegers'
            raise TypeError(msg.format(cls = cls))

In [32]:
# 完整代码
from array import array
import math
import reprlib
import numbers

class Vector:
    typecode = 'd'

    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):     # 如果参数是 slice 对象
            return cls(self._components[index]) # 返回新的类
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be initegers'
            raise TypeError(msg.format(cls = cls))

    def __init__(self, components) -> None:
        # self._components是受保护的实例属性，把 Vector 分量保存在一个数组中
        self._components = array(self.typecode, components)
    
    def __iter__(self): # 将实例变成可迭代的对象
        return iter(self._components)
    
    def __repr__(self) -> str:
        components = reprlib.repr(self._components) # reprlib.repr() 可以让输出太多的时候使用 ... 省略一部分
        components = components[components.find('['):-1] # 先去掉前面的 array('d' 和 后面的 )
        return 'Vector({})'.format(components)

    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):  # 直接计算平方和
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod    # 直接传入 memoryview 给构造方法，而不是拆包
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [33]:
v7 = Vector(range(7))
v7[-1]

6.0

In [34]:
v7[1:3]

Vector([1.0, 2.0])

# 10.5 Vector 类第 3 版：动态存取属性
Vector2d 变成 Vector 后，就没办法通过名称访问向量分量了(如 v.x 和 v.y)。现在我们处理的向量可能有大量分量，不过，若能通过单个字母访问几个分量的话会很方便  
在 Vector2d 中，我们使用 @property 装饰器把 x 和 y标记为只读特性，我们可以在 Vector 中编写四个特效，但这样太麻烦  
特殊方法 \_\_getattr__ 提供了更好的方式

属性查找失败后，解释器会调用 \_\_getattr__ 方法。简单来说，对 my_obj.x 表达式，Python 会检查 my_obj 实例有没有名为 x 的属性，如果没有就到类 (my_obj.\_\_class__) 中查找。如果还没有，会顺着继承树继续查找。如果依旧找不到，调用 my_obj 所属类定义的 \_\_getattr__ 方法，传入 self 和属性名称的字符串形式(如 'x')

In [ ]:
# 支持 Vector 查找 xyzt 属性
shortcut_names = 'xyzt'

def __getattr__(self, name):
    cls = type(self)
    if len(name) == 1:
        pos = cls.shortcut_names.find(name)
        if 0 <= pos < len(self._components):
            return self._components[pos]
    msg = '{.__name__!r} object is has no attribute {!r}'
    raise AttributeError(msg.format(cls, name))


但这样会出现一个问题，当用户为实例赋值时，会添加给真正的 v.x 后续则不会有限调用 _getattr__  
为了避免这种情况，我们要改写 Vector 中设置属性的逻辑

In [ ]:
def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:  # 特别处理名称为单字符的属性
        if name in cls.shortcut_names:
            # 对 xyzt 名称特殊输出
            error = 'readonly attribute {attr_name!r}'
        elif name.islower():
            # 对小写字母设置输出
            error = "can't set attributes 'a' to 'z' in {cls_name!r}"
        else:
            error = ''
        if error:
            msg = error.format(cls_name = cls.__name__,attr_name = name)
            raise AttributeError(msg)
    super().__setattr__(name, value)    # 在超类调用__setattr__方法，提供标准行为

# 10.6 Vector 类第 4 版：散列和快速等值测试
我们要再次实现 \_\_hash__ 方法，加上现有的 \_\_eq__ 方法，这会吧 Vector 实例变成可散列的对象  
上次我们只对两个值使用异或计算，而这次我们需要对每一个值的hash进行异或，所以这里可以用到归约函数 reduce  
funtools.reduce() 的原理：把一系列值归约成单个值。reduce() 函数的第一个参数是接受两个参数的函数，第二个参数是一个可迭代对象。加入有个接受两个参数的 fn 函数和一个 lst 列表，调用 reduce(fn, lst) 时，fn 会应用到第一对元素上，即 fb(lst[0], lst[1])，生成第一个结果r1，然后，fn 会应用到 r1 到下一个元素上，即 fn(r1, lst[2])，生成 r2，以此类推，直到最后一个元素，返回结果 rn

In [ ]:
#计算整数 0~5 的累计异或的 3 种方式
n = 0
for i in range(1, 6):
    n ^= i

#
import functools
functools.reduce(lambda a, b:a^b, range(1, 6))

#
import operator
functools.reduce(operator.xor , range(1, 6))

In [36]:
# vector_v4.py 部分代码，在上一个版本 的基础上导入两个模块，添加 __hash__ 方法
from array import array
import reprlib, math, functools, operator

class Vector:
    typecode = 'd'

    # 省略 ...

    def __eq__(self, other) -> bool:
        return tuple(self) == tuple(other)
    
    def __hash__(self) -> int:
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)    # 注意 0 是初始值

In [37]:
help(functools.reduce)

Help on built-in function reduce in module _functools:

reduce(...)
    reduce(function, sequence[, initial]) -> value
    
    Apply a function of two arguments cumulatively to the items of a sequence,
    from left to right, so as to reduce the sequence to a single value.
    For example, reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]) calculates
    ((((1+2)+3)+4)+5).  If initial is present, it is placed before the items
    of the sequence in the calculation, and serves as a default when the
    sequence is empty.



使用 reduce 函数时最好提供第三个参数，reduce(function, sequence[, initial])，这样能避免这个异常 TypeError: reduce() of empty sequence with no initial value  
如果序列为空，initial 应是返回的结果；否则，在归约中使用它作为第一个参数，因此应该使用恒等值。比如 对 +、| 和 ^ 来说，应使用 0 作为 initial，而对 * 或 & 来说，使用 1

这种实现方法是以中映射归约计算：把函数应用到各个元素上，生成一个新序列（映射，map），然后计算整个值（归约，reduce）

In [ ]:
# 使用 map 实现的 __hash__
def __hash__(self):
    hashes = map(hash,self._components)
    return functools.reduce(operator.xor, hashes, 0)

优化 \_\_eq__ 方法，减少处理时间和内存用量，而且对于之前的实现 Vector([1, 2]) 和 (1, 2) 是相等的，这或许是个问题。

In [ ]:
# 提高效率的 __eq__ 方法
def __eq__(self, other):
    if len(self) != len(other):
        return False
    for a,b in zip(self, other):
        if a != b:
            return False
    return True

上面的实现效率不错，但用于计算聚合值的整个 for 循环可以替换成一行 all 函数调用：如果所有分量比对效果是 True，那就返回 True，否则返回 False

In [ ]:
def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a,b in zip(self, other))

注：zip 和 enumerate 生成器函数在循环体中很有用

# 10.7 Vector 类第 5 版：格式化
Vector 类的 \_\_format__ 方法与 Vector2d 类相似，但不适用极坐标，而使用球面坐标（也称超球面坐标），因为它可能有很多维度，超过四维后，球体变成了超球体，因此我们将自定义格式中的后缀由 'p' 变为 'h'  
例如，对四维空间 (len(v) == 4) 中的 Vector 对象来说，'h' 代码所得到的结果是这样：<r, Φ1, Φ2, Φ3>。其中，r 是模 (abs(v))，余下三个是角坐标。

与上次一样，我们需要定义两个辅助方法：一个是 angle(n)，用于计算某个角坐标;另一个是angles()，返回由所有角坐标构成的可迭代对象。

In [ ]:
import itertools
class Vector:
    typecode = 'd'

    # 省略
    # ...

    # 使用 n 维球体公式计算某个角坐标
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
    
    # 创建生成器表达式，按需计算所有角坐标
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec: str) -> str:
        if fmt_spec.endswith('h'):  # 超球面坐标
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())    # 使用 itertools.chain 函数生成生成器表达式，无缝迭代向量的模和各个角坐标
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))